In [ ]:
import pandas as pd
import numpy as np
import gp_lib

In [ ]:
import importlib
gp_lib = importlib.reload(gp_lib)

In [ ]:
df = pd.read_csv('sotonmet.txt')

In [ ]:
t_tmp = pd.to_numeric(pd.to_datetime(df['Reading Date and Time (ISO)']))/int(1e9)
t_tmp -= t_tmp[0]
y_tmp = df['Tide height (m)']
data = pd.concat([t_tmp, y_tmp], axis=1)

In [ ]:
data.to_csv('extracted_data.csv',index=False)

In [ ]:
# visualise the data to determine sensible hparams
import plotly.express as px
px.scatter(data, x='Reading Date and Time (ISO)', y='Tide height (m)')

In [ ]:
# get nan rows:
test = data[~pd.notna(y_tmp)].to_numpy()
train = data[pd.notna(y_tmp)].to_numpy()

In [ ]:
X,y = train[:,0], train[:,1]
Xt = test[:,0]

In [ ]:
scal = gp_lib.Scalar()
scal.fit(X,y)

In [ ]:

X,y = scal.transform(X,y)
Xt, _ = scal.transform(Xt,y)

In [ ]:
regr = gp_lib.GaussianProcess().fit(X,y)
yt, vart = regr.predict(Xt)
Xt,yt = scal.reverse(Xt,yt)
vart = scal.y_std**2*vart

In [ ]:
px.scatter(x=Xt,y=np.zeros_like(Xt))

In [ ]:
data_cp = data.copy(deep=True)
data_cp.loc[~pd.notna(y_tmp),'Tide height (m)'] = yt

In [ ]:
px.scatter(data_cp,x='Reading Date and Time (ISO)',y='Tide height (m)')